In [92]:
import pandas as pd
import json
from bs4 import BeautifulSoup
import requests

In [3]:
from ScopusScrapus import ScopusSearchQuery

AUTHOR = 'jason karlawish'
params = {'query': f'AUTHOR-NAME({AUTHOR})', 'view': 'COMPLETE', 'count': '3', 'httpAccept': 'application/json'}
key = '260dbeeac318b7268a18b520c3691743'
ssq = ScopusSearchQuery(key,params)

In [6]:
with open("Scopus.json", "w") as file:
    for paper in ssq:
        json.dump(paper, file)
        file.write("\n")

In [93]:
data_SCOPUS = list()
with open("Scopus.json", "r") as file:
    for paper in file:
        paper_json = json.loads(paper)
        paper_json["affiliation-country"] = paper_json["affiliation"][0]["affiliation-country"]
        junk_columns = ["@_fa", "link", "prism:url", "prism:pageRange", "prism:volume", "prism:issueIdentifier",
                        "prism:coverDisplayDate", "affiliation", "openaccess", "openaccessFlag", "freetoread", "freetoreadLabel",
                        "eid"]
        for junk in junk_columns:
            if (junk in paper_json.keys()):
                paper_json.pop(junk)
        data_SCOPUS.append(paper_json)

In [94]:
df_scopus = pd.DataFrame(data_SCOPUS)
df_scopus.head()

,dc:identifier,dc:title,dc:creator,prism:publicationName,prism:issn,prism:eIssn,prism:coverDate,prism:doi,citedby-count,pubmed-id,prism:aggregationType,subtype,subtypeDescription,article-number,source-id,affiliation-country,pii
0,SCOPUS_ID:85193647431,Caregiver Accounts of Lucid Episodes in Person...,Karlawish J.,Gerontologist,00169013,17585341,2024-06-01,10.1093/geront/gnad170,0,38134428,Journal,ar,Article,gnad170,29070,United States,NaN
1,SCOPUS_ID:85146365004,Clinical Practice in the Alzheimer Biomarker E...,Karlawish J.,JAMA Network Open,NaN,25743805,2023-01-13,10.1001/jamanetworkopen.2022.50938,0,36637828,Journal,no,Note,NaN,21100900345,United States,NaN
2,SCOPUS_ID:85138247748,Importance of Asking Older Adults Whether They...,Karlawish J.,JAMA Network Open,NaN,25743805,2022-01-01,10.1001/jamanetworkopen.2022.31442,0,36098971,Journal,no,Note,NaN,21100900345,United States,NaN
3,SCOPUS_ID:85113347140,Aducanumab and the Business of Alzheimer Disea...,Karlawish J.,JAMA Neurology,21686149,21686157,2021-11-01,10.1001/jamaneurol.2021.3123,6,34410302,Journal,no,Note,NaN,21100228519,United States,NaN
4,SCOPUS_ID:85110708275,The approval of Aduhelm risks eroding public t...,Karlawish J.,Nature Reviews Neurology,17594758,17594766,2021-09-01,10.1038/s41582-021-00540-6,48,34267383,Journal,no,Note,NaN,17700156408,United States,NaN


In [ ]:
#Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/123.0.0.0 Safari/537.36
#Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:124.0) Gecko/20100101 Firefox/124.0
#Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/123.0.0.0 Safari/537.36 Edg/123.0.2420.81
#Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/123.0.0.0 Safari/537.36 OPR/109.0.0.0
#Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/123.0.0.0 Safari/537.36
#Mozilla/5.0 (Macintosh; Intel Mac OS X 14.4; rv:124.0) Gecko/20100101 Firefox/124.0
#Mozilla/5.0 (Macintosh; Intel Mac OS X 14_4_1) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/17.4.1 Safari/605.1.15
#Mozilla/5.0 (Macintosh; Intel Mac OS X 14_4_1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/123.0.0.0 Safari/537.36 OPR/109.0.0.0
#Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/123.0.0.0 Safari/537.36
#Mozilla/5.0 (X11; Linux i686; rv:124.0) Gecko/20100101 Firefox/124.0

In [120]:
headers = {"User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/123.0.0.0 Safari/537.36"}
issn_list = df_scopus["prism:issn"].dropna().unique()

with open("SJR.json", "w") as file:
  for issn in issn_list:
    url = f"https://www.scimagojr.com/journalsearch.php?q={issn}"
    response = requests.get(url=url, headers=headers)
    soup = BeautifulSoup(response.content, 'html.parser')
    page_id = soup.find("div", class_="search_results").a.get("href")
    journal_url = f"https://www.scimagojr.com/{page_id}"
    journal_response = requests.get(url=journal_url, headers=headers)
    journal_soup = BeautifulSoup(journal_response.content, 'html.parser')

    journal_name = journal_soup.find("h1").contents[0].strip()
    journal_publisher = journal_soup.find("div", class_="journalgrid").find_all("div")[2].a.string.strip()
    journal_issn = issn
    journal_thematic_scope = journal_soup.find("div", class_="journalgrid").find_all("div")[1].p.find("li").a.string.strip()
    journal_H_index = journal_soup.find("div", class_="journalgrid").find_all("div")[3].p.string.strip()
    journal_Quartile = journal_soup.find("div", class_="dashboard").find_all("tr")[-1].find_all("td")[-1].string.strip()
    journal_SJR = journal_soup.find_all("div", class_="dashboard")[1].find("table").find_all("tr")[-1].find_all("td")[1].string.strip()
    journal_Impact_Factor = journal_soup.find_all("div", class_="dashboard")[1].find_all("table")[2].find_all("tr")[-1].find_all("td")[2].string.strip()

    journal = {
        "ISSN": journal_issn,
        "Name": journal_name,
        "Publisher": journal_publisher,
        "Theme": journal_thematic_scope,
        "H-Index": journal_H_index,
        "Quartile": journal_Quartile,
        "SJR": journal_SJR,
        "Impact_Factor": journal_Impact_Factor
    }

    json.dump(journal, file)
    file.write("\n")

In [121]:
data_SJR = list()

with open("SJR.json", "r") as file:
    for journal in file:
        journal_json = json.loads(journal)
        data_SJR.append(journal_json)

In [124]:
df_SJR = pd.DataFrame(data_SJR)
df_SJR.head()

,ISSN,Name,Publisher,Theme,H-Index,Quartile,SJR,Impact_Factor
0,00169013,"Gerontologist, The",Oxford University Press,Medicine,161,Q1,1.913,5.444
1,21686149,JAMA Neurology,American Medical Association,Medicine,272,Q1,6.194,8.858
2,17594758,Nature Reviews Neurology,Nature Publishing Group,Medicine,207,Q1,6.237,10.606
3,21686106,JAMA Internal Medicine,American Medical Association,Medicine,390,Q1,4.363,4.705
4,15265161,American Journal of Bioethics,Taylor and Francis Ltd.,Medicine,71,Q1,0.967,1.188
